In [1]:
import subprocess,sys
from keras import backend as K
from keras_retinanet import models
import tensorflow as tf
import cv2,os,datetime
import numpy as np
import time
from keras_retinanet.utils.visualization import draw_box, draw_caption
import pandas as pd
import tensorflow.contrib.tensorrt as trt
import IPython.display as Disp

from keras_retinanet import models

Using TensorFlow backend.


In [2]:
t = datetime.datetime.now()
file_name = t.strftime("%d_%m_%Y_%H_%M_%S_")

##### setting file paths

In [3]:
####Get inference model
model_name = "resnet50_csv_28.h5"
retinanet_snapshot = "../../../dataset/helmet_n_vest/snapshots/"+model_name
inference_model = "../../../model_with_worker/05_06_2019_09_24_40_resnet50_csv_50.pb"
inference_model_tensorflow = "../../../model_with_worker/05_06_2019_09_24_40_resnet50_csv_50.pb"

In [4]:
def snapshot_to_inference_retinanet(input_model_name,output_model_name,env='cv_p35'):
    ! retinanet-convert-model {input_model_name} {output_model_name}
    

In [5]:
def convert_snapshot_to_protobuf(input_model_name, output_model_name,save_graph_def=False):
    !{sys.executable} keras_to_tensorflow.py --input_model={input_model_name}  --output_model={output_model_name} --save_graph_def={save_graph_def}
#     subprocess.call(['python keras_to_tensorflow.py --input_model="../../retinanet/snapshots_1/resnet50_csv_19.h5"  --output_model="model.pb"'])


In [6]:
def get_graph(frozen_model_file):
    """
    returns graphdef(unserialized) object
    """
    with tf.gfile.GFile(frozen_model_file,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

In [7]:
retinanet_snapshot = '../model_files/Retinanet/tf_model/'
inference_model = '/home/prateek/Downloads/Notebooks/abg/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'
inference_model_tensorflow = '../model_files/Retinanet/tf_model/resnet50_coco_best_v2.1.0.pb'

In [ ]:
snapshot_to_inference_retinanet(retinanet_snapshot, inference_model)

In [8]:
convert_snapshot_to_protobuf(inference_model, inference_model_tensorflow,save_graph_def=True)
Disp.clear_output(wait=True)

Using TensorFlow backend.
W0817 16:39:34.717434 139787627129728 deprecation_wrapper.py:119] From /home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 16:39:38.252744 139787627129728 deprecation_wrapper.py:119] From /home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 16:39:41.022494 139787627129728 deprecation_wrapper.py:119] From /home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0817 16:39:43.930248 139787627129728 deprecation_wrapper.py:119] From /home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.ran

I0817 16:40:41.754323 139787627129728 graph_util_impl.py:311] Froze 306 variables.
I0817 16:40:41.913398 139787627129728 graph_util_impl.py:364] Converted 306 variables to const ops.
I0817 16:40:43.185979 139787627129728 keras_to_tensorflow.py:163] Saved the freezed graph at ../model_files/Retinanet/tf_model/resnet50_coco_best_v2.1.0.pb


##### creating tensorrt model

In [4]:
model_input = 'input_1:0'
model_output = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
               'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
               'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

In [7]:
precision = "FP16"
batch_size=10
workspace_size=1 << 27
trt_graph = trt.create_inference_graph(get_graph(inference_model_tensorflow),model_output,\
                                       precision_mode=precision,max_workspace_size_bytes=workspace_size,\
                                      max_batch_size=batch_size,is_dynamic_op=True)

with tf.gfile.GFile(inference_model_tensorflow.replace('inference_model','tensorrt_model').replace('.pb','_precision='+precision+'_batchsize='+str(batch_size)+'_trt.pb'),'wb') as f:
    f.write(trt_graph.SerializeToString())

INFO:tensorflow:Running against TensorRT version 5.0.2


In [9]:
# check how many ops of the original frozen model
all_nodes = len([1 for n in get_graph(inference_model_tensorflow).node])
print("numb. of all_nodes in frozen graph:", all_nodes)

# check how many ops that is converted to TensorRT engine
trt_engine_nodes = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("numb. of trt_engine_nodes in TensorRT graph:", trt_engine_nodes)
all_nodes = len([1 for n in trt_graph.node])
print("numb. of all_nodes in TensorRT graph:", all_nodes)

numb. of all_nodes in frozen graph: 2011
numb. of trt_engine_nodes in TensorRT graph: 56
numb. of all_nodes in TensorRT graph: 1322
